In [69]:
import numpy as np
import pandas as pd
from hampel._filters import hampel
from scipy.ndimage import median_filter

In [70]:
fs = 20000
freq = 13.47
impulse_intervals = 287
time = 2
dc_amp = 3.1
arr_len = fs * time
win_len = 101
thresh = 3
t = np.arange(arr_len) / fs
np.random.seed(42)
x = t * dc_amp + np.sin(t * 2 * np.pi * freq) + np.random.randn(arr_len)
x[::impulse_intervals] += np.random.randn(arr_len)[::impulse_intervals] * 5

In [71]:
x_hampel, median, mad = hampel(x, win_size=win_len, thresh=thresh)

In [72]:
median_ref = median_filter(x, size=win_len, mode='nearest')

In [73]:
%timeit median_filter(x, size=win_len, mode='nearest')

47.8 ms ± 405 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [74]:
%timeit hampel(x, win_size=win_len, thresh=thresh)

24.6 ms ± 731 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [75]:
x_ped = np.concatenate([np.ones(win_len) * x[0], x, np.ones(win_len) * x[-1]])
mad_ref = pd.Series(x_ped).rolling(window=win_len, center=True).apply(lambda x: np.median(np.abs(x - np.median(x))))[win_len:-win_len].values
x_hampel_ref = np.where(np.abs(x - median_ref) > thresh * mad_ref, median_ref, x)

In [76]:
(median==median_ref).all() and (mad==mad_ref).all() and (x_hampel==x_hampel_ref).all()

True